In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [15]:
nestdf = pd.read_csv('Nest.csv')
partavgdf = pd.read_csv('Part_Avg.csv')
partdf = pd.read_csv('Part.csv')

In [16]:
joined_nest_partsavg_df = pd.merge(nestdf, partavgdf, on='ixJobSummary', how='inner')
joined_nest_partsavg_df #avg num of parts and nest data in one df

,ixJobSummary,fStrategies,dLengthUsed_Avg,dWidthUsed_Avg,dPartArea_Job,dTrueAreaRectified_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist
0,130032,0,240.000000,95.990000,519.0404,23037.600000,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,145075,8,108.453240,39.263200,18389.9746,21573.573062,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891
2,233110,8,38.709400,35.736100,879.9165,1383.322989,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600
3,286531,8,69.701700,60.558550,5992.2342,9448.707993,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000
4,504531,0,6.807000,38.006900,198.8594,258.712968,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,2583832,0,143.770400,94.319500,3562.2303,13560.352243,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000
1790932,548877,4,34.513950,55.300350,2898.6813,4526.656120,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000
1790933,2420784,8,83.969400,33.398500,1732.8873,2804.452006,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000
1790934,2302130,8,80.844067,54.065133,7208.6328,14136.765442,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650


In [17]:
joined_nest_partsavg_df['CropUtil'] = joined_nest_partsavg_df['dPartArea_Job'] / joined_nest_partsavg_df['dTrueAreaRectified_Job']

joined_nest_partsavg_df.drop('dLengthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dWidthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dTrueAreaRectified_Job', axis=1, inplace=True)


joined_nest_partsavg_df.drop('ixJobSummary', axis=1, inplace=True)

joined_nest_partsavg_df

,fStrategies,dPartArea_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist,CropUtil
0,0,519.0404,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022530
1,8,18389.9746,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891,0.852431
2,8,879.9165,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600,0.636089
3,8,5992.2342,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000,0.634186
4,0,198.8594,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.768649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0,3562.2303,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000,0.262695
1790932,4,2898.6813,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000,0.640358
1790933,8,1732.8873,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000,0.617906
1790934,8,7208.6328,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650,0.509921


In [9]:
#joined_nest_partsavg_df.drop('ixJobSummary', axis=1, inplace=True)

In [10]:
#joined_nest_partsavg_df['CropUtil'] = joined_nest_partsavg_df['dPartArea_Job'] / joined_nest_partsavg_df['dTrueAreaRectified_Job']

# Print the DataFrame to verify the changes
#joined_nest_partsavg_df

,fStrategies,dLengthUsed_Avg,dWidthUsed_Avg,dPartArea_Job,dTrueAreaRectified_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist,CropUtil
0,0,240.000000,95.990000,519.0404,23037.600000,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022530
1,8,108.453240,39.263200,18389.9746,21573.573062,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891,0.852431
2,8,38.709400,35.736100,879.9165,1383.322989,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600,0.636089
3,8,69.701700,60.558550,5992.2342,9448.707993,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000,0.634186
4,0,6.807000,38.006900,198.8594,258.712968,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.768649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0,143.770400,94.319500,3562.2303,13560.352243,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000,0.262695
1790932,4,34.513950,55.300350,2898.6813,4526.656120,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000,0.640358
1790933,8,83.969400,33.398500,1732.8873,2804.452006,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000,0.617906
1790934,8,80.844067,54.065133,7208.6328,14136.765442,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650,0.509921


In [18]:
# Separate data by strategy
strategy_groups = joined_nest_partsavg_df.groupby('fStrategies')

In [19]:
# Placeholder for models and scalers
models = {}
scalers = {}
encoders = {}

In [20]:
# Define an autoencoder function
def create_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(encoding_dim, activation='relu')(input_layer)
    decoder = Dense(input_dim, activation='sigmoid')(encoder)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    encoder_model = Model(inputs=input_layer, outputs=encoder)
    return autoencoder, encoder_model

In [21]:
from sklearn.metrics import mean_absolute_error

In [22]:
for strategy, group in strategy_groups:
    X = group.drop(columns=['CropUtil', 'fStrategies'])
    y = group['CropUtil']

    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Define and train the autoencoder
    input_dim = X_train.shape[1]
    encoding_dim = 10  # Adjust as needed
    autoencoder, encoder_model = create_autoencoder(input_dim, encoding_dim)
    autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

    # Encode the features
    X_train_encoded = encoder_model.predict(X_train)
    X_test_encoded = encoder_model.predict(X_test)

    # Train a regression model
    rf = RandomForestRegressor() #used to predict utilization for strategy with encoded values.
    rf.fit(X_train_encoded, y_train)

    # Evaluate the model
    y_pred = rf.predict(X_test_encoded)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Strategy {strategy} - Random Forest MSE: {mae}')

    # Store the models and scalers
    models[strategy] = rf
    scalers[strategy] = scaler
    encoders[strategy] = encoder_model

Epoch 1/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 6s 547us/step - loss: 0.7708 - val_loss: 0.6765
Epoch 2/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 6s 592us/step - loss: 0.8787 - val_loss: 0.6748
Epoch 3/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 6s 594us/step - loss: 0.7277 - val_loss: 0.6745
Epoch 4/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 6s 603us/step - loss: 0.6891 - val_loss: 0.6744
Epoch 5/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 7s 703us/step - loss: 1.5650 - val_loss: 0.6743
Epoch 6/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 7s 670us/step - loss: 0.8780 - val_loss: 0.6743
Epoch 7/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 6s 560us/step - loss: 0.7284 - val_loss: 0.6743
Epoch 8/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 6s 568us/step - loss: 0.6489 - val_loss: 0.6742
Epoch 9/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 7s 641us/step - loss: 0.6285 - val_loss: 0.6742
Epoch 10/10
10416/10416 ━━━━━━━━━━━━━━━━━━━━ 6s 557us/step - loss: 0.5921 - val_loss: 0.6742
13020/13020 ━━━━━━━━━━━━━━━━━━━━ 5s 356us/step
3255/3255 ━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# Predict utilization for new data using all strategies
def predict_utilization_all_strategies(new_data):
    results = {}
    for strategy in models.keys():
        scaler = scalers[strategy]
        encoder = encoders[strategy]
        model = models[strategy]
        
        # Standardize and encode new data
        X_new_scaled = scaler.transform(new_data)
        X_new_encoded = encoder.predict(X_new_scaled)
        predictions = model.predict(X_new_encoded)
        results[strategy] = predictions
    return results

In [ ]:
# Provided data
provided_data = pd.DataFrame({
    'dLengthUsed_Avg': [240.000000, 108.453240, 38.709400],
    'dWidthUsed_Avg': [95.990000, 39.263200, 35.736100],
    'dPartArea_Job': [519.0404, 18389.9746, 879.9165],
    'dTrueAreaRectified_Job': [23037.600000, 21573.573062, 1383.322989],
    'dLength_Avg': [11.214286, 38.347309, 37.702200],
    'dWidth_Avg': [7.588371, 12.843400, 20.209600],
    'dArea_Avg': [58.148629, 475.076800, 439.958200],
    'cNested_Avg': [1, 4, 2],
    'fExtShape_Avg': [9, 5, 0],
    'dExtArea_Avg': [0.000000, 496.283591, 439.958200],
    'dExtBoundaryDist_Avg': [0.000000, 6.026536, 9.161500],
    'dExtContainedDist_Avg': [0.000000, 39.581636, 36.520700],
    'dLgIntArea_Avg': [0.000000, 14.593373, 0.000000],
    'dLgIntBoundaryDist_Avg': [0.000000, 0.535145, 0.000000],
    'dLgIntContainedDist_Avg': [0.000000, 2.846391, 0.000000],
    'dLgExtConArea_Avg': [0.000000, 12.247745, 60.162600],
    'dLgExtConBoundaryDist': [0.000000, 0.341673, 2.761000],
    'dLgExtConContainedDist': [0.000000, 0.728891, 12.675600]
})

# Predict utilization for the provided data using all strategies
pred_util_all_strategies = predict_utilization_all_strategies(provided_data)
for strategy, pred_util in pred_util_all_strategies.items():
    print(f'Predicted utilization for {strategy}: {pred_util}')

In [ ]:
top_strategies = []
for i in range(len(provided_data)):
    # Get predictions for the current row across all strategies
    row_predictions = {strategy: pred_util[i] for strategy, pred_util in pred_util_all_strategies.items()}
    # Sort strategies by predicted utilization
    sorted_strategies = sorted(row_predictions.items(), key=lambda item: item[1], reverse=True)
    # Get the top 3 strategies
    top_3 = sorted_strategies[:3]
    top_strategies.append(top_3)

# Print the top 3 strategies for each row
for idx, top_3 in enumerate(top_strategies):
    print(f'Entry {idx + 1}:')
    for strategy, util in top_3:
        print(f'  Strategy: {strategy}, Predicted Utilization: {util}')